## Powerco

Powerco has three electricity power plants that supply the needs of four cities.
- The supply capacity of each power plant and the demand in each city (in million
KWH/hour) are given in the table below.

| From/to | City 1 | City 2 | City 3 | City 4 | Supply |
| --- | --- | --- | --- | --- | --- |
| Plant 1 | $8 | $6 | $10 | $9 | 35 |
| Plant 2 | $9 | $12 | $13 | $7 | 50 |
| Plant 3 | $14 | $9 | $16 | $5 | 40 |
| Demand | 45 | 20 | 30 | 30 |

- The cost of sending 1 million KWH of electricity from each plant to each city is
also given the table.

**Question:** How does Powerco make its distribution plan to minimize the total cost
while satisfying all cities’ demands?


### Data:

In [1]:
numplants = 3
numcities = 4

costs = [8 6 10 9;
        9 12 13 7;
        14 9 16 5]
        
supply = [35 50 40]
demand = [45 20 30 30];

### Model:

In [2]:
using JuMP, HiGHS

power = Model(HiGHS.Optimizer)

#Create a variable xij for each pair of plant and city that represents
#the amount of power sent from plant i to city j
@variable(power, x[1:numplants,1:numcities] >= 0)

#Each power plant i can provide at most supply[i] power
@constraint(power, supplyconstraint[i in 1:numplants], sum(x[i,j] for j in 1:numcities) <= supply[i])

#Each city j requires at least demand[j] power
@constraint(power, demandconstraint[j in 1:numcities], sum(x[i,j] for i in 1:numplants) >= demand[j])

#Cost of sending from plant i to city j is given by costs[i,j]
@objective(power, Min, sum(sum(costs[i,j]*x[i,j] for j in 1:numcities) for i in 1:numplants))

print(power)

Min 8 x[1,1] + 6 x[1,2] + 10 x[1,3] + 9 x[1,4] + 9 x[2,1] + 12 x[2,2] + 13 x[2,3] + 7 x[2,4] + 14 x[3,1] + 9 x[3,2] + 16 x[3,3] + 5 x[3,4]
Subject to
 

demandconstraint[1] : x[1,1] + x[2,1] + x[3,1] ≥ 45
 demandconstraint[2] : x[1,2] + x[2,2] + x[3,2] ≥ 20
 demandconstraint[3] : x[1,3] + x[2,3] + x[3,3] ≥ 30
 demandconstraint[4] : x[1,4] + x[2,4] + x[3,4] ≥ 30
 supplyconstraint[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] ≤ 35
 supplyconstraint[2] : x[2,1] + x[2,2] + x[2,3] + x[2,4] ≤ 50
 supplyconstraint[3] : x[3,1] + x[3,2] + x[3,3] + x[3,4] ≤ 40
 x[1,1] ≥ 0
 x[2,1] ≥ 0
 x[3,1] ≥ 0
 x[1,2] ≥ 0
 x[2,2] ≥ 0
 x[3,2] ≥ 0
 x[1,3] ≥ 0
 x[2,3] ≥ 0
 x[3,3] ≥ 0
 x[1,4] ≥ 0
 x[2,4] ≥ 0
 x[3,4] ≥ 0


In [3]:
optimize!(power)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
7 rows, 12 cols, 24 nonzeros
7 rows, 12 cols, 24 nonzeros
Presolve : Reductions: rows 7(-0); columns 12(-0); elements 24(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(125) 0s
          7     1.0200000000e+03 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  1.0200000000e+03
HiGHS run time      :          0.00


In [4]:
@show objective_value(power)
@show value.(x)

objective_value(power) = 1020.0
value.(x) = 

[0.0 10.0 25.0 0.0; 45.0 0.0 5.0 0.0; 0.0 10.0 0.0 30.0]


3×4 Matrix{Float64}:
  0.0  10.0  25.0   0.0
 45.0   0.0   5.0   0.0
  0.0  10.0   0.0  30.0